In [45]:
# import everything
import spacy
from spacy.matcher import Matcher
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import requests
from flair.models import SequenceTagger
flair_12class = SequenceTagger.load('ner-ontonotes-fast')
flair_4class = SequenceTagger.load('ner')

2020-11-21 20:27:47,031 loading file /home/sandipan/.flair/models/en-ner-ontonotes-fast-v0.4.pt
[nltk_data] Downloading package punkt to /home/sandipan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sandipan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/sandipan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/sandipan/nltk_data...
[nltk_data]   Package words is already up-to-date!
2020-11-21 20:27:51,471 loading file /home/sandipan/.flair/models/en-ner-conll03-v0.4.pt


In [19]:
document= "so now the fake news nytimes is trace the coronavirus origin back to europe not china this is a first i wonder what the fail new york times get for this one are there any name source they were recently throw out of china like dog and obviously want back in sad"

In [20]:
# load spacy models
en = None
def get_spacy_model():
    global en
    global _model
    if not en:
       _model = spacy.load('en_core_web_lg')
    en = _model
    return en

In [21]:
# Get all NER using SPACY
def get_ner_spacy(text):
    text = 'NULL' if text is None else str(text)
    nlp = get_spacy_model()
    ner_tags = {}
    if text:        
        doc = nlp(text)
        for ee in doc.ents:
            ner_tags[ee.text] = ee.label_
    return ner_tags

In [22]:
# run the spacy code
a = get_ner_spacy(document)
print(a)

{'nytimes': 'ORG', 'europe': 'LOC', 'china': 'GPE', 'first': 'ORDINAL', 'new york times': 'ORG'}


In [23]:
# get all NOUNs and proper nouns POS tag using SPACY
def get_noun_spacy(text):
    text = 'NULL' if text is None else str(text)
    nlp = get_spacy_model()
    noun_tags = {}
    if text:        
        doc = nlp(text)
        for token in doc:
            if(token.pos_ == 'NOUN' or token.pos_ == 'PROPN'):
                noun_tags[token.text] = token.pos_
    return noun_tags

In [24]:
# run the code
a = get_noun_spacy(document)
print(a)

{'news': 'NOUN', 'nytimes': 'PROPN', 'trace': 'NOUN', 'coronavirus': 'NOUN', 'origin': 'NOUN', 'europe': 'PROPN', 'china': 'PROPN', 'new': 'PROPN', 'york': 'PROPN', 'times': 'NOUN', 'one': 'NOUN', 'name': 'NOUN', 'source': 'NOUN', 'dog': 'PROPN'}


In [26]:
# Subtree Matching for Relation Extraction using spacy
def subtree_matcher(text):
  nlp = get_spacy_model()
  doc = nlp(text)
  
  subjpass = 0

  for i,tok in enumerate(doc):
    # find dependency tag that contains the text "subjpass"    
    if tok.dep_.find("subjpass") == True:
      subjpass = 1

  x = {}
  y = {}
  v = {}

  # if subjpass == 1 then sentence is passive
  if subjpass == 1:
    for i,tok in enumerate(doc):
      if tok.dep_.find("subjpass") == True:
        y[tok.dep_] = tok.text

      if tok.dep_.endswith("obj") == True:
        x[tok.dep_] = tok.text
  
  # if subjpass == 0 then sentence is not passive
  else:
    for i,tok in enumerate(doc):
      if tok.dep_.endswith("subj") == True:
        x[tok.dep_] = tok.text

      if tok.dep_.endswith("obj") == True:
        y[tok.dep_] = tok.text
  for i,tok in enumerate(doc):
      if tok.dep_.endswith("ROOT") == True:
          v[tok.dep_] = tok.text  
  return x,v,y

In [27]:
# run Subtree Matching
a = subtree_matcher(document)
print(a)

({'dobj': 'what', 'pobj': 'sad'}, {'ROOT': 'is'}, {'nsubjpass': 'they'})


In [31]:
# use GATE API call
# import requests
# url = "https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer"
# headers = {'Content-Type': 'text/plain'}
# response = requests.post(url, data=document, headers=headers).json()

# import json
# print(json.dumps(response, indent=2))

In [46]:
def gate_ner(sentence):
  return [(sentence[entity["indices"][0]:entity["indices"][1]] + f" ({entity['gender']})",entity_type) if entity_type == "Person" and "gender" in entity else (sentence[entity["indices"][0]:entity["indices"][1]],entity_type)  for entity_type,entities in requests.post("https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer", data=sentence, headers={'Content-Type': 'text/plain'}).json()["entities"].items() for entity in entities]

In [33]:
# GATE NER using API
gate_ner(document)

[('china', 'Location'),
 ('china', 'Location'),
 ('new york times', 'Organization')]

In [35]:
# NLTK
def nltk_ner(document):
  return {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [37]:
nltk_ner(document)

set()

In [47]:
#flair
# having dependency issues
def flair_ner(document, model):
  from flair.data import Sentence
  s = Sentence(document)
  model.predict(s)
  entities = s.to_dict(tag_type='ner')
  return [(entity["text"], entity["type"]) for entity in entities["entities"]]

In [48]:
# flair_ner(document, flair_4class)

KeyError: 'type'

In [44]:
# flair_ner(document, flair_12class)

KeyError: 'type'

In [11]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern1 = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]
pattern2 = [{"LOWER": "hello"}, {"LOWER": "world"}]

matcher.add("HelloWorld", None, pattern1,pattern2)
# matcher.add("HelloWorld", None,
#             [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}],
#             [{"LOWER": "hello"}, {"LOWER": "world"}])

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 3 Hello, world
15578876784678163569 HelloWorld 4 6 Hello world


In [14]:
# Get all NER
import spacy
nlp = spacy.load('en_core_web_lg')
document= "so now the fake news nytimes is trace the coronavirus origin back to europe not china this is a first i wonder what the fail new york times get for this one are there any name source they were recently throw out of china like dog and obviously want back in sad"
def get_ner_spacy(text):
    if text:
        doc = nlp(text)
        ner_tags = {}
        for ee in doc.ents:
            ner_tags[ee.text] = ee.label_
        return ner_tags

# run the code
a = get_ner_spacy(document_string)
print(a)

{'nytimes': 'ORG', 'europe': 'LOC', 'china': 'GPE', 'first': 'ORDINAL', 'new york times': 'ORG'}


In [17]:
# get all NOUNs and proper nouns
doc = nlp(document)
for token in doc:
    if(token.pos_ == 'NOUN' or token.pos_ == 'PROPN'):
        print(token.text, token.pos_)

news NOUN
nytimes PROPN
trace NOUN
coronavirus NOUN
origin NOUN
europe PROPN
china PROPN
new PROPN
york PROPN
times NOUN
one NOUN
name NOUN
source NOUN
china PROPN
dog PROPN


In [18]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

nytimes 21 28 ORG
europe 69 75 LOC
china 80 85 GPE
first 96 101 ORDINAL
new york times 125 139 ORG
china 215 220 GPE


NameError: name 'nlp' is not defined

In [86]:
print(subtree_matcher(doc))

({'dobj': 'what', 'pobj': 'sad'}, {'ROOT': 'is'}, {'nsubjpass': 'they'})


In [5]:
document= "so now the fake news nytimes is trace the coronavirus origin back to europe not china this is a first i wonder what the fail new york times get for this one are there any name source they were recently throw out of china like dog and obviously want back in sad"
import requests
url = "https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer"
headers = {'Content-Type': 'text/plain'}
response = requests.post(url, data=document, headers=headers).json()
import json
print(json.dumps(response, indent=2))

{
  "text": "so now the fake news nytimes is trace the coronavirus origin back to europe not china this is a first i wonder what the fail new york times get for this one are there any name source they were recently throw out of china like dog and obviously want back in sad",
  "entities": {
    "Location": [
      {
        "indices": [
          80,
          85
        ],
        "locType": "country",
        "rule": "Location1",
        "ruleFinal": "LocFinal",
        "matches": [
          143,
          145
        ]
      },
      {
        "indices": [
          215,
          220
        ],
        "locType": "country",
        "rule": "Location1",
        "ruleFinal": "LocFinal",
        "matches": [
          143,
          145
        ]
      }
    ],
    "Organization": [
      {
        "indices": [
          125,
          139
        ],
        "orgType": "company",
        "rule": "GazOrganization",
        "ruleFinal": "OrgFinal"
      }
    ]
  }
}


In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize("so now the fake news nytimes is trace the coronavirus origin back to europe not china this is a first i wonder what the fail new york times get for this one are there any name source they were recently throw out of china like dog and obviously want back in sad"))) if hasattr(chunk, 'label') }

[nltk_data] Downloading package punkt to /home/sandipan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sandipan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/sandipan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/sandipan/nltk_data...
[nltk_data]   Package words is already up-to-date!


set()

In [9]:
!pip3 install -U git+https://github.com/aboSamoor/polyglot.git@master
!polyglot download embeddings2.en ner2.en
from polyglot.text import Text
Text(document).entities

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-b0us1o72
     |████████████████████████████████| 233 kB 7.2 MB/s 
    ERROR: Command errored out with exit status 1:
     command: '/home/sandipan/Insync/ghoshm21@gmail.com/Google Drive/personal_project/Learning_NLP/nlp/bin/python3' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-3ogsz3jz/pyicu/setup.py'"'"'; __file__='"'"'/tmp/pip-install-3ogsz3jz/pyicu/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-vkle08a1
         cwd: /tmp/pip-install-3ogsz3jz/pyicu/
    Complete output (53 lines):
    (running 'icu-config --version')
    (running 'pkg-config --modversion icu-i18n')
    Traceback (most recent call last):
      File "/tmp/pip-install-3ogsz3jz/pyicu/setup.py", line 63, in <module>
        ICU_VERS

ModuleNotFoundError: No module named 'polyglot'